In [1]:
import kodsp as dsp
import json
import pandas as pd

openai_key = 'YOUR_API_KEY'

Not loading Cohere because it is not installed.
Not loading Cohere because it is not installed.


In [2]:
def convert_data_to_dsp_v1(data_type):
    dsp_examples = []
    with open(f"./KorWikiTQ_ko_{data_type}.json", "r") as json_file:
        korwikitq = json.load(json_file)['data']
    for idx, datum in enumerate(korwikitq):
        dsp_examples.append(dsp.Example(question=datum['QAS']['question'], 
                                        answer=datum['QAS']['answer']))
    return dsp_examples

# LM

In [5]:
lm = dsp.GPT3(model='gpt-3.5-turbo', api_key=openai_key, model_type='chat')

dsp.settings.configure(lm=lm)

# Data

In [3]:
train = convert_data_to_dsp_v1(data_type='train')
dev = convert_data_to_dsp_v1(data_type='dev')

In [13]:
# dsp.Example 형태 확인

train[0], dev[0]

({'question': '2010년 8월 11일 대한민국 축구 국가대표팀 은퇴식을 거행한 선수는 누구인가요?',
  'answer': '이운재'},
 {'question': '나익진은 몇 대 체신부 차관을 지냈나요?', 'answer': '7대'})

In [14]:
# 원하는 task에 맞게 prefix, desc, instructions만 간단하게 수정해주면 끝입니다!

Question = dsp.Type(prefix="질문: ", desc="${표와 관련된 질문}")
Answer = dsp.Type(prefix="정답: ", desc="${표에서 찾아낸 질문에 대한 답}", format=dsp.format_answers)

qa_template_v1 = dsp.Template(instructions="주어진 질문에 대해 답하세요.",
                              question=Question(),
                              answer=Answer())

# Vanilla LM

In [7]:
def vanilla_LM_QA(question: str) -> str:
    demos = dsp.sample(train, k=5)
    example = dsp.Example(question=question, demos=demos)

    example, completions = dsp.generate(qa_template_v1)(example, stage='qa')
    return completions.answer

In [8]:
vanilla_LM_QA(dev[5].question)
lm.inspect_history(n=1)





주어진 질문에 대해 답하세요.

---

출력 형식은 다음과 같습니다.

질문: ${표와 관련된 질문}
정답: ${표에서 찾아낸 질문에 대한 답}

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

질문: 손승우가 요도 역을 연기한 드라마는 뭔가요?
정답:'미스터 션샤인'





In [9]:
vanilla_LM_QA("DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?")
lm.inspect_history(n=1)





주어진 질문에 대해 답하세요.

---

출력 형식은 다음과 같습니다.

질문: ${표와 관련된 질문}
정답: ${표에서 찾아낸 질문에 대한 답}

---

질문: 농림부 3대 차관보은 누구입니까?
정답: 구윤석(具允錫)

질문: 브느와 델롬이 2005년에 연출한 작품은 뭔가요?
정답: 프로포지션

질문: 아라라트 예레반의 1975-76 시즌 UEFA 컵위너스컵 1라운드 최종 점수가 어떻게 돼?
정답: 10-1

질문: 애니메이션 박앵귀 10화의 부제를 일본어로 뭐라고 하나요?
정답: 絆のゆくえ

질문: 비주얼 스튜디오 6.0의 코드명이 뭐야?
정답: Aspen

질문: DSBA 연구실에서 4학기를 다니고 있으며, 성이 오씨인 학생의 생일은?
정답:정답을 알 수 없습니다. 추가 정보가 필요합니다.



